In [71]:
import os
import json
import numpy as np
import pandas as pd
from archetypal import settings
from pathlib import Path
from ladybug.epw import EPW
from uuid import uuid4
from shoeboxer.builder import ShoeboxConfiguration, ShoeBox
# from shoeboxer.schedules import default_schedules
from utils.constants import *
import torch
from lightning.pytorch import Trainer
from torch.utils.data import DataLoader
from ml.surrogate import Surrogate
from ml.predict import predict_ubem
from weather.weather import extract

# Check if we are running on Windows or Linux using os
if os.name == "nt":
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("C:/EnergyPlusV22-2-0")
else:
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("/usr/local/EnergyPlus-22-2-0")

In [72]:
registry = "ml-for-building-energy-modeling/model-registry"
model = "Global UBEM Shoebox Surrogate with Combined TS Embedder"
tag = "v3"
resource = "model.ckpt"

surrogate = Surrogate.load_from_registry(registry, model, tag, resource)
surrogate.model.eval()

trainer = Trainer(
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_progress_bar=True,
    )

c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'target_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['target_transform'])`.
c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'weather_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['weather_transform'])`.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, u

In [73]:
# Get grasshopper json data for config
gh_data_dir = "D:/DATA/SingleShoeboxTests/Shoebox.json"
with open(gh_data_dir, 'r') as f:
    gh_data = json.load(f)

In [74]:
gh_data["ShoeboxConfig"]["ground_2_footprint"]

0

In [75]:
epw_path = "C:/ProgramData/Solemma/Common/WeatherData/EPW/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2004-2018.epw"
# epw_path ="C:/ProgramData/Solemma/Common/WeatherData/EPW/USA_NY_New.York-LaGuardia.AP.725030_TMYx.2004-2018.epw"
if gh_data["Conditioning"]["HeatRecoveryType"] == "None":
    hrv = 0
else:
    hrv = HRV[gh_data["Conditioning"]["HeatRecoveryType"]].value 
config = {
    "FacadeMass": 0, #TODO
    "RoofMass": 0,
    "FacadeRValue": 4.88,
    "RoofRValue": 4.88,
    "SlabRValue": 4.88,
    "WindowUValue": gh_data["SimpleWindow"]["uval"],
    "WindowShgc": gh_data["SimpleWindow"]["shgc"],
    "PeopleDensity": gh_data["Loads"]["PeopleDensity"],
    "EquipmentPowerDensity": gh_data["Loads"]["EquipmentPowerDensity"],
    "LightingPowerDensity": gh_data["Loads"]["LightingPowerDensity"],
    "HeatingSetpoint": gh_data["Conditioning"]["HeatingSetpoint"],
    "CoolingSetpoint": gh_data["Conditioning"]["CoolingSetpoint"],
    "cop_heating": 1,
    "cop_cooling": 1,
    "VentilationMode": MechVentMode[gh_data["Conditioning"]["MechVentSchedule"]].value,
    "VentilationPerArea": gh_data["Conditioning"]["MinFreshAirArea"]/1000,
    "VentilationPerPerson": gh_data["Conditioning"]["MinFreshAirPerson"]/1000,
    "EconomizerSettings": Econ[gh_data["Conditioning"]["EconomizerType"]].value,
    "RecoverySettings": hrv,
    "Infiltration": gh_data["Infiltration"]["InfiltrationFlowPerExteriorSurfaceArea"],
    "wwr": gh_data["ShoeboxConfig"]["wwr"],
    "roof_2_footprint": gh_data["ShoeboxConfig"]["roof_2_footprint"],
    "ground_2_footprint": gh_data["ShoeboxConfig"]["ground_2_footprint"],
    "orientation": gh_data["ShoeboxConfig"]["orientation"],
    "width": gh_data["ShoeboxConfig"]["width"],
    "height": gh_data["ShoeboxConfig"]["height"],
    "perim_depth": gh_data["ShoeboxConfig"]["perim_depth"],
    "core_depth": gh_data["ShoeboxConfig"]["core_depth"],
    "building_idx": 0,
    "edge_length": 1,
    "edge_weight": 1,
    "weight": 1,
    "template_idx": 0.0,
    "floor_name": "none",
}

In [76]:
"""
Make Schedules
"""
schedules = pd.read_hdf("data/schedules.h5")
idx = gh_data["ShoeboxConfig"]["schedule_index"]
scheds = schedules.loc[schedules.index[idx][0]].to_numpy()

In [77]:
# schedules.xs('EquipmentAvailabilitySchedule', level=1, drop_level=True).to_csv("EquipmentAvailabilitySchedule.csv")
# schedules.xs('LightsAvailabilitySchedule', level=1, drop_level=True).to_csv("LightsAvailabilitySchedule.csv")
# schedules.xs('OccupancySchedule', level=1, drop_level=True).to_csv("OccupancySchedule.csv")

In [78]:
"""
Make Shoebox
"""
sb_config = ShoeboxConfiguration()
sb_config.width = config["width"]
sb_config.height = config["height"]
sb_config.adiabatic_partition_flag = 0
sb_config.perim_depth = config["perim_depth"]
sb_config.core_depth = config["core_depth"]
sb_config.wwr = config["wwr"]
sb_config.roof_2_footprint = config["roof_2_footprint"]
sb_config.ground_2_footprint = config["ground_2_footprint"]
sb_config.shading_vect = np.zeros(12)
sb_config.orientation = config["orientation"]
template_dict = config.copy()
template_dict["VentilationMode"] = 0
template_dict["schedules"] = scheds

sb = ShoeBox(
    name=str(uuid4()),
    epw=epw_path,
    shoebox_config=sb_config,
    template_dict=template_dict,
    output_directory="data/sim_results/"
)
idf = sb.idf(run_simulation=False)

"""
Simulate
"""

hourly_df, monthly_df = sb.simulate(idf=idf)
df = sb.postprocess(monthly_df)
annual = df.T.groupby(level=[0,1]).sum().T
annual

INFO:shoeboxer.geometry_utils:New perimeter depth: 3.0
INFO:shoeboxer.geometry_utils:New core depth: 7.0
INFO:shoeboxer.geometry_utils:Shifting core origin to perimeter edge; moving -2.0 m in y axis
INFO:shoeboxer.geometry_utils:Updating adiabatic surfaces.
INFO:shoeboxer.geometry_utils:Maximum shading point is 0.0
INFO:shoeboxer.geometry_utils:Placing shading around center point [1.5 0. ]
INFO:ShoeBox:Building idf for data\sim_results\fc5c124c-bb56-4277-9196-eeea71b876b9.epjson
INFO:ShoeBox:EnergyPlus Starting
INFO:ShoeBox:EnergyPlus, Version 22.2.0-c249759bad, YMD=2023.11.18 20:27
INFO:ShoeBox:Converted input file format. Exiting.
INFO:ShoeBox:EnergyPlus Run Time=00hr 00min  0.12sec
INFO:ShoeBox:Command executed successfully.
C:\EnergyPlusV22-2-0\ExpandObjects.EXE #0-fc5c124c-bb56-4277-9196-eeea71b876b9.idf: 3.00it [00:00, 16.2it/s]
C:\EnergyPlusV22-2-0\energyplus.exe fc5c124c-bb56-4277-9196-eeea71b876b9.idf: 55it [00:04, 11.13it/s]


KeyValue   Name   
Core       Cooling     3.485333
           Heating     8.467618
Perimeter  Cooling     9.074352
           Heating    20.165615
dtype: float64

In [79]:
# core, heating, cooling
out_file = gh_data_dir.split(".json")[0] + "_whitebox_out.csv"
print(out_file)
np.savetxt(out_file, np.resize(df.to_numpy(), (4,12)), delimiter=",")


D:/DATA/SingleShoeboxTests/Shoebox_whitebox_out.csv


In [80]:
climate = extract(EPW(epw_path))
for i in range(12):
    config[f"shading_{i}"] = 0
features = pd.DataFrame(data=config, index=[0])

In [81]:
from ml.data import PredictBuildingDataset

dataset = PredictBuildingDataset(features, np.expand_dims(scheds, 0),climate, surrogate.space_config)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
shoebox_predictions = trainer.predict(surrogate, dataloader)
shoebox_predictions = torch.cat(shoebox_predictions)
try:
    shoebox_predictions = shoebox_predictions.cpu().numpy()
except:
    shoebox_predictions = shoebox_predictions.numpy()
shoebox_predictions[shoebox_predictions <= 0] = 0
shoebox_predictions = pd.DataFrame(
    shoebox_predictions,
    columns=surrogate.target_transform.columns,
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\zoelh\AppData\Local\anaconda3\envs\ml-for-bem\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 66.67it/s]


In [82]:
import plotly.express as px
print(annual)
print(shoebox_predictions.T.groupby(level=[0,1]).sum().T)

KeyValue   Name   
Core       Cooling     3.485333
           Heating     8.467618
Perimeter  Cooling     9.074352
           Heating    20.165615
dtype: float64
KeyValue      Core            Perimeter           
Name       Cooling    Heating   Cooling    Heating
0         3.636428  14.731413  7.223319  26.271744


In [83]:
# core, heating, cooling
out_file = gh_data_dir.split(".json")[0] + "_surrogate_out.csv"
print(out_file)
np.savetxt(out_file, np.resize(shoebox_predictions.to_numpy(), (4,12)), delimiter=",")

D:/DATA/SingleShoeboxTests/Shoebox_surrogate_out.csv
